In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
import os

sys.path.append(os.path.abspath("/data/home/ngjhn/read"))


In [1]:
from transformers import VisionEncoderDecoderModel
from transformers.adapters import LoRAConfig, BertAdapterModel, ModelAdaptersMixin, ModelWithFlexibleHeadsAdaptersMixin, ViTAdapterModel



config = LoRAConfig(r=8)

# freeze(text_model)
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    "google/vit-base-patch16-224-in21k", "bert-base-uncased")
# freeze(model)

# for p in model.parameters():
#     p.requires_grad = False
model.decoder.add_adapter("lora_adapter", config=config, set_active=True)
model.encoder.add_adapter("lora_adapter", config=config, set_active=True)

# text_model.add_causal_lm_head("lm_head")


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-u

In [1]:
!nvidia-smi

Thu Apr 20 00:10:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:90:1C.0 Off |                    0 |
| N/A   62C    P0   279W / 400W |  39643MiB / 40960MiB |     99%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from coco import *
from adapter_utils import *
from utils import *

data_module = CocoDataModule(
    data_dir="/data/home/ngjhn/read/coco",
    ann_dir="/data/home/ngjhn/read/coco/annotations/",
    vision_model_name_or_path="google/vit-base-patch16-224-in21k",
    split_file="/data/home/ngjhn/read/coco/dataset_coco.json",
    collate_fn=collate_fn,
    train_batch_size=1,
    val_batch_size=128)

pl_model = ImageCaptioningModel("google/vit-base-patch16-224-in21k",
                                "bert-base-uncased",
                                data_module=data_module,
                                learning_rate=1e-4,
                                peft_conf=create_config("rnn",
                                                        hidden_dim=256,
                                                        prefix_length=10))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.9.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.9.crossattention.self.query.bias', 'bert.encoder.layer.11.crossattention.self.value.bias', 'bert.encoder.layer.11.crossattention.output.dense.weight', 'bert

Enables rnn-adapter tuning; num trainable params 2,234,880, ratio 0.016
loading annotations into memory...
Done (t=0.62s)
creating index...
index created!
loading annotations into memory...
Done (t=0.32s)
creating index...
index created!
loading annotations into memory...
Done (t=0.31s)
creating index...
index created!


In [8]:
batch = next(iter(data_module.train_dataloader()))

In [ ]:
python3 coco.py --slurm_folder $(pwd)/slurm --name "rnn 512 bert large" --ft_type rnn --sweep --learning_rate 0 --hidden_dim 512

In [11]:
pl_model.training_step(batch=batch, batch_idx=0)

/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/pytorch_lightning/core/module.py:411: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


{'loss': tensor(13.4318, grad_fn=<NllLossBackward0>)}